In [1]:
using SDDP
using GLPK

In [2]:
thermal_plants = [1,2,3,4]
c0 = 2.592
ρ = 0.96
ci = [10, 20, 40, 80]
cd = 500
m = 1000
vi = 2050
vmax = 4100
qmax = 1500
ptmax = [100,150,200,250]

4-element Vector{Int64}:
 100
 150
 200
 250

In [3]:
graph = SDDP.LinearGraph(3)

# SDDP.add_node(graph,:2)
# SDDP.add_node(graph,:3)
# SDDP.add_node(graph,:4)
# SDDP.add_node(graph,:5)
# SDDP.add_node(graph,:6)
# SDDP.add_node(graph,:7)

# SDDP.add_edge(graph, 1 => 2, 0.5)
# SDDP.add_edge(graph, 1 => 3, 0.5)
# SDDP.add_edge(graph, 2 => 4, 0.5)
# SDDP.add_edge(graph, 2 => 5, 0.5)
# SDDP.add_edge(graph, 3 => 6, 0.5)
# SDDP.add_edge(graph, 3 => 7, 0.5)

graph

Root
 0
Nodes
 1
 2
 3
Arcs
 0 => 1 w.p. 1.0
 1 => 2 w.p. 1.0
 2 => 3 w.p. 1.0


In [5]:
model = SDDP.PolicyGraph(
    graph,
    sense = :Min,
    lower_bound = 0.0,
    optimizer = GLPK.Optimizer,
) do subproblem, node
    # State variables
    @variable(subproblem, 0 <= volume <= vmax, SDDP.State, initial_value = vi)
    # Control variables
    @variables(subproblem, begin
        thermal_generation[thermal_plants] >= 0
        0 <= hydro_generation <= qmax
        hydro_spill >= 0
        load_shedding >= 0
    end)
    # Random variables
    if node == 1
        @variable(subproblem, inflow)
        Ω = [150.0]
        P = [1.0]
        SDDP.parameterize(subproblem, Ω, P) do ω
            return JuMP.fix(inflow, ω)
        end
    elseif node == 2
        @variable(subproblem, inflow)
        Ω = [450.0,300.0]
        P = [0.5,0.5]
        SDDP.parameterize(subproblem, Ω, P) do ω
            return JuMP.fix(inflow, ω)
        end
    else
        @assert node == 3
        @variable(subproblem, inflow)
        Ω = [771.0,213.0]
        P = [0.5,0.5]
        SDDP.parameterize(subproblem, Ω, P) do ω
            return JuMP.fix(inflow, ω)
        end
    end
    
    # Transition function and constraints
    @constraints(
        subproblem,
        begin
            volume.out == volume.in - c0*(hydro_generation + hydro_spill) + c0*inflow
            demand_constraint, sum(thermal_generation[i] for i in thermal_plants) + load_shedding + ρ*hydro_generation == m
            thermal_generation_max_constraint[t in thermal_plants],thermal_generation[t] <= ptmax[t]
        end
    )
    # Stage-objective
    @stageobjective(subproblem, sum(ci[i]*thermal_generation[i] for i in thermal_plants) + cd*load_shedding)
end

A policy graph with 3 nodes.
 Node indices: 1, 2, 3


In [6]:
SDDP.train(model; iteration_limit = 10)

------------------------------------------------------------------------------
                      SDDP.jl (c) Oscar Dowson, 2017-21

Problem
  Nodes           : 3
  State variables : 1
  Scenarios       : 4.00000e+00
  Existing cuts   : false
  Subproblem structure                      : (min, max)
    Variables                               : (11, 11)
    VariableRef in MOI.LessThan{Float64}    : (2, 3)
    AffExpr in MOI.LessThan{Float64}        : (4, 4)
    VariableRef in MOI.GreaterThan{Float64} : (9, 9)
    AffExpr in MOI.EqualTo{Float64}         : (2, 2)
Options
  Solver          : serial mode
  Risk measure    : SDDP.Expectation()
  Sampling scheme : SDDP.InSampleMonteCarlo

Numerical stability report
  Non-zero Matrix range     [1e+00, 3e+00]
  Non-zero Objective range  [1e+00, 5e+02]
  Non-zero Bounds range     [2e+03, 4e+03]
  Non-zero RHS range        [1e+02, 1e+03]
No problems detected

 Iteration    Simulation       Bound         Time (s)    Proc. ID   # Solves
        

In [7]:
simulations = SDDP.simulate(
    # The trained model to simulate.
    model,
    # The number of replications.
    100,
    # A list of names to record the values of.
    [:volume, :thermal_generation, :hydro_generation, :hydro_spill],
);

In [14]:
volumes_dic = Dict{Vector{Float64},Vector{Float64}}();
simulations_used = []
println("======================DISTINCT SIMULATIONS======================")
for s in 1:100
    nodes = []
    volumes = []
    outgoing_volume = map(simulations[s]) do node
        append!(nodes,node[:node_index])
        append!(volumes,round(node[:volume].out,digits=2))
    end
    if !(haskey(volumes_dic, volumes))
        push!(volumes_dic, volumes => nodes)
        append!(simulations_used, s)
        println("Simulation $(s) - nodes: $(nodes[1]) -> $(nodes[2]) -> $(nodes[3]) , volumes: ($(round(volumes[1],digits=2)), $(round(volumes[2],digits=2)), $(round(volumes[3],digits=2)))")
    end
end

======================DISTINCT SIMULATIONS======================
Simulation 1 - nodes: 1 -> 2 -> 3 , volumes: (953.8, 635.2, 0.0)
Simulation 2 - nodes: 1 -> 2 -> 3 , volumes: (953.8, 257.9, 0.0)


In [15]:
println("======================DISTINCT SIMULATIONS======================")
for s in simulations_used
    println("\nSimulation $(s)")
    map(simulations[s]) do node
        println("Node $(node[:node_index]), hydro_generation - $(round(node[:hydro_generation],digits=2)), thermal_generations - T1: $(round(node[:thermal_generation][1],digits=2)), T2: $(round(node[:thermal_generation][2],digits=2)), T3: $(round(node[:thermal_generation][3],digits=2)), T4: $(round(node[:thermal_generation][4],digits=2))") 
    end
end

======================DISTINCT SIMULATIONS======================

Simulation 1
Node 1, hydro_generation - 572.92, thermal_generations - T1: 100.0, T2: 150.0, T3: 200.0, T4: 0.0
Node 2, hydro_generation - 572.92, thermal_generations - T1: 100.0, T2: 150.0, T3: 200.0, T4: 0.0
Node 3, hydro_generation - 1016.06, thermal_generations - T1: 24.58, T2: 0.0, T3: 0.0, T4: 0.0

Simulation 2
Node 1, hydro_generation - 572.92, thermal_generations - T1: 100.0, T2: 150.0, T3: 200.0, T4: 0.0
Node 2, hydro_generation - 568.48, thermal_generations - T1: 100.0, T2: 150.0, T3: 200.0, T4: 4.26
Node 3, hydro_generation - 870.5, thermal_generations - T1: 100.0, T2: 64.32, T3: 0.0, T4: 0.0


In [16]:
objectives = map(simulations) do simulation
    return sum(stage[:stage_objective] for stage in simulation)
end

μ, ci = SDDP.confidence_interval(objectives)
println("Confidence interval: ", μ, " ± ", ci)
println("Lower bound: ", SDDP.calculate_bound(model))

Confidence interval: 38789.611851851856 ± 2642.702055342753
Lower bound: 38008.69629629631
